# Deploy Function in WML

In [1]:
import os
import wml_sdk_utils as wml_util

In [2]:
WML_SPACE_ID = '934341dc-0a71-4d86-9c09-a47261359cca'

In [3]:
wml_client = wml_util.get_client(space_id=WML_SPACE_ID)
wml_client.version

'1.0.286'

In [14]:
wml_client.repository.list_functions()

------------------------------------  --------------------------------------------  ------------------------  ------  -----------  -------------------
GUID                                  NAME                                          CREATED                   TYPE    SPEC_STATE   SPEC_REPLACEMENT
25ce5086-ac88-4a8e-93ec-dab1439f6ce3  generic metrics wendy                         2023-09-08T14:27:26.002Z  python  supported
69c52d18-4f9a-4f8e-805b-687e8f8372cd  wml dummy function dima                       2023-09-08T14:25:47.002Z  python  supported
4b5ce464-6460-4780-b585-3d4160e4daa9  wml dummy function carlin                     2022-05-11T19:11:08.002Z  python  unsupported  runtime-22.2-py3.10
95176c20-64f9-41e7-8804-78d8912864af  Segmentation Metrics Provider Function wendy  2022-05-11T17:32:30.002Z  python  unsupported  runtime-22.2-py3.10
------------------------------------  --------------------------------------------  ------------------------  ------  -----------  ------------

#### input schema
`{"input_data":[{"values":[<something>]}]}`
- the input has to have a field "input_data"
- the list passed to "values" cannot be empty
- you can have more name-value pairs as you wish, for example:
```
{"input_data": [{"values":[None],"myparam": "abc"}]}
{"input_data": [{"values":[None]}], "myparam": "abc"}
```


#### output schema
`{"predictions":[]}`
- the output has to have a field "predictions"
- you can add your own name-value pairs as you wish similar to the input schema:
```
{"predictions":["myparam":"abc"]}
{"predictions":[],"myparam":"abc"}
```

## Deploy a Dummy Function

In [5]:
path_script = 'wml_deployable_function_dummy.py'
function_name = 'wml dummy function dima'
deploy_name = 'wml dummy function deploy dima'

In [6]:
wml_client.service_instance._href_definitions.get_functions_href()

'https://internal-nginx-svc.cpd.svc.cluster.local:12443/ml/v4/functions'

In [7]:
function_asset_id = wml_util.function_store(path_script,wml_client,function_name=function_name)
print(function_asset_id)

Deleted deployment wml dummy function deploy dima (ec591e05-6548-401f-a0d5-6f804b9f4d13) associated with asset wml dummy function dima (c44f7b8d-c984-4860-b946-a8d637cc3903).
Deleted function asset wml dummy function dima (c44f7b8d-c984-4860-b946-a8d637cc3903).
b56101f1-309d-549b-a849-eaa63f77b2fb
Function UID = 69c52d18-4f9a-4f8e-805b-687e8f8372cd
deleting tmp/wml_deployable_function_dummy.py.gz...
69c52d18-4f9a-4f8e-805b-687e8f8372cd


In [8]:
deployment_id,scoring_url = wml_util.function_deploy(function_asset_id,wml_client,function_deployment_name=deploy_name)
print(deployment_id,scoring_url)



#######################################################################################

Synchronous deployment creation for uid: '69c52d18-4f9a-4f8e-805b-687e8f8372cd' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b4ba1fa5-295a-4f40-9417-edc020a49d8a'
------------------------------------------------------------------------------------------------


b4ba1fa5-295a-4f40-9417-edc020a49d8a https://internal-nginx-svc.cpd.svc.cluster.local:12443/ml/v4/deployments/b4ba1fa5-295a-4f40-9417-edc020a49d8a/predictions?version=2023-09-08


In [9]:
import requests
import json

headers = {'Authorization':'Bearer '+os.environ['USER_ACCESS_TOKEN'],
           'Content-Type':'application/json'}

payload = {'input_data':[{'values':[None]}],
           'batch_id':1,
           'batch_label':114}

res = requests.post(url=scoring_url,
                    data=json.dumps(payload),
                    headers=headers,
                    verify=False)
res

<Response [400]>

In [10]:
res.content

b'{"trace":"7b32dbee947da8b796a1adde9e26df82","errors":[{"code":"score_processing_failure","message":"[Errno 2] No such file or directory: \'data/cifar-10-batches-py/data_batch_1\'"}],"status_code":400}'

## Deploy A Dummy Custom Metric Provider

In [11]:
path_script = 'custom_metrics_dummy.py'
function_name = 'generic metrics wendy'
deploy_name = 'generic metrics deploy wendy'

In [12]:
function_asset_id = wml_util.function_store(path_script,wml_client,function_name=function_name)
print(function_asset_id)

Deleted deployment generic metrics deploy wendy (c452ff9b-51a0-48eb-8e7a-ff2ab07e7b83) associated with asset generic metrics wendy (041d8b87-8316-4cf0-86a4-e19f925270fd).
Deleted function asset generic metrics wendy (041d8b87-8316-4cf0-86a4-e19f925270fd).
b56101f1-309d-549b-a849-eaa63f77b2fb
Function UID = 25ce5086-ac88-4a8e-93ec-dab1439f6ce3
deleting tmp/custom_metrics_dummy.py.gz...
25ce5086-ac88-4a8e-93ec-dab1439f6ce3


In [13]:
deployment_id,scoring_url = wml_util.function_deploy(function_asset_id,wml_client,function_deployment_name=deploy_name)
print(deployment_id,scoring_url)



#######################################################################################

Synchronous deployment creation for uid: '25ce5086-ac88-4a8e-93ec-dab1439f6ce3' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
.......
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='ea9882a8-9a2e-40bf-a0c6-e8a50ec93764'
------------------------------------------------------------------------------------------------


ea9882a8-9a2e-40bf-a0c6-e8a50ec93764 https://internal-nginx-svc.cpd.svc.cluster.local:12443/ml/v4/deployments/ea9882a8-9a2e-40bf-a0c6-e8a50ec93764/predictions?version=2023-09-08


#### Sadly you cannot really test this endpoint until you have a real openscale subscription.